In [ ]:
import json
from functools import lru_cache

In [ ]:
from collections import defaultdict

import ipywidgets as widgets
import requests

tor_list = requests.get(
    "https://raw.githubusercontent.com/SecOps-Institute/Tor-IP-Addresses/master/tor-exit-nodes.lst"
).text.splitlines()
tor_list = [i.strip() for i in tor_list]
print(f"{len(tor_list)} Node Ips Loaded")


def isittor(ip):
    if ip in tor_list:
        return True
    else:
        return False

In [ ]:
from geolite2 import geolite2


@lru_cache()
def get_country(ip):
    try:
        if isittor(ip):
            return "TOR"
        x = geo.get(ip)
    except ValueError:
        return None
    try:
        return x["country"]["names"]["en"] if x else None
    except KeyError:
        return None


geo = geolite2.reader()

In [ ]:
new_ip_out = widgets.Output(layout={"border": "1px solid black"})
country_out = widgets.Output(layout={"border": "1px solid tan"})
all_country_out = widgets.Output(layout={"border": "1px solid orange"})

In [ ]:
import sys
import time

import libtorrent as lt

In [ ]:
ses = lt.session({"listen_interfaces": "0.0.0.0:6881"})

info = lt.torrent_info("./Your Torrent File.torrent")
h = ses.add_torrent({"ti": info, "save_path": "./download"})
s = h.status()

In [ ]:
additional_trackers = [
    s.strip()
    for s in requests.get(
        "https://raw.githubusercontent.com/ngosang/trackerslist/master/trackers_best_ip.txt"
    )
    .text.replace("\n\n", "\n")
    .splitlines()
]
additional_trackers

In [ ]:
bad_trackers = []
for t in h.trackers():
    if t["last_error"]["value"] == 0:
        print(t["last_error"]["value"])

In [ ]:
[h.add_tracker({"url": url}) for url in additional_trackers]
print("Adding Trackers")

In [ ]:
all_ips = []

In [ ]:
new_ip_out

In [ ]:
country_out

In [ ]:
all_country_out

In [ ]:
print("starting", s.name)
start = time.time()
h.force_dht_announce()
h.force_reannounce()
while not s.is_seeding:
    s = h.status()
    if time.time() - start > 10:
        start = time.time()
        h.force_dht_announce()
        h.force_reannounce()
    print(f"\r{s.progress*100}% tp: {len(all_ips)} {s.state}", end=" ")

    alerts = ses.pop_alerts()
    for a in alerts:
        if a.category() & lt.alert.category_t.error_notification:
            print(a)
    with new_ip_out:
        for peer in h.get_peer_info():
            if peer.ip[0] not in all_ips:
                all_ips.append(peer.ip[0])
                with open("all_ips2.json", "w") as f:
                    f.write(json.dumps(all_ips, indent=4))
                print(peer.ip[0])
    country_out.clear_output()
    with country_out:
        c = defaultdict(int)
        for peer in h.get_peer_info():
            c[get_country(peer.ip[0])] += 1
        for item in sorted(c.items(), key=lambda item: item[1], reverse=True):
            print(item[0], item[1])

    sys.stdout.flush()

    time.sleep(1)
print(h.status().name, "download complete")

In [ ]:
start2 = time.time()
while s.is_seeding:
    s = h.status()
    print(
        f"down: {s.download_rate / 1000} kB/s up: {s.upload_rate / 1000} kB/s | active: {s.num_peers} | total known: {len(all_ips)}) ",
        end="\r",
    )

    if time.time() - start > 15:
        start = time.time()
        h.force_dht_announce()
        h.force_reannounce()
    if time.time() - start2 > 20:
        start2 = time.time()
        all_country_out.clear_output()
        with all_country_out:
            print(f"total discovered {len(all_ips)}")
            c = defaultdict(int)
            for peer in all_ips:
                c[get_country(peer)] += 1
            for item in sorted(c.items(), key=lambda item: item[1], reverse=True):
                print(item[0], item[1])
    alerts = ses.pop_alerts()
    for a in alerts:
        if a.category() & lt.alert.category_t.error_notification:
            print(a)
    for peer in h.get_peer_info():
        if peer.ip[0] not in all_ips:
            all_ips.append(peer.ip[0])
            with open("all_ips2.json", "w") as f:
                f.write(json.dumps(all_ips, indent=4))
            with new_ip_out:
                print(peer.ip[0])
    country_out.clear_output()
    """ with country_out:
        c = defaultdict(int)
        for peer in h.get_peer_info():
            c[get_country(peer.ip[0])] += 1
        for item in sorted(c.items(), key=lambda item: item[1],reverse=True):
            print(item[0], item[1])"""

    time.sleep(1)

In [ ]:
with open("all_ips2.json", "w") as f:
    f.write(json.dumps(all_ips, indent=4))

In [ ]:
!pip3 install pytorrent
!pip3 install maxminddb-geolite2

In [ ]:
all_ips